<a href="https://colab.research.google.com/github/FireStrings/MasterDegree/blob/main/UtilsNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libs import

In [ ]:
import numpy as np
import pandas as pd
from pandas.errors import SettingWithCopyWarning
from pandas.plotting import autocorrelation_plot
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from scipy.special import expit
from scipy.stats import zscore
from scipy import stats

import os
import io
import math
from os import listdir
from os import system
import subprocess
from functools import reduce

import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from time import time
from pytz import timezone, all_timezones_set, common_timezones_set
import pytz

import random
import tensorflow as tf

import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)


%matplotlib inline


### Functions

In [ ]:
def manage_results(_file, op, results=None):
    if op == "read":
        with open(_file, 'rb') as f:
            loaded_dict = pickle.load(f)
            return loaded_dict
    else:
        with open(_file, "wb") as f:
            pickle.dump(results, f)

def manage_models(_file, op, results=None):
    if op == "read":
        with open(_file, 'rb') as f:
            loaded_model = pickle.load(f)
            return loaded_model
    else:
        with open(_file, "wb") as f:
            pickle.dump(results, f)

def define_seed(seed):
  np.random.seed(seed)
  tf.random.set_seed(seed)
  random.seed(seed)
  tf.config.experimental.enable_op_determinism()

def select_and_normalize_minmax(anual_df, list_cols):
    df = anual_df[list_cols]

    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

    return df_scaled.fillna(df_scaled.mean())

def select_and_normalize_divided(anual_df, list_cols):

    def divided_by_1000(n):
        return n/1000

    df = anual_df[list_cols]

    df_scaled = df.apply(divided_by_1000)

    return df_scaled.fillna(df_scaled.mean())

def set_num_rows_cols(nr, nc):
    pd.set_option('display.max_columns', nc)
    pd.set_option('display.max_rows', nr)

def set_size_plot(x, y):
    sns.set_theme(rc={'figure.figsize':(x, y)})

def getSigmoid():
    arr = np.arange(0, 1, 0.02)
    np.hstack(np.vstack(arr))
    def sigmoid(x):
        return 1/(1+expit(-x))

    sigmoid(arr)
    sns.scatterplot(x=arr, y=sigmoid(arr))

def load_files(station, preproc=True):

    raw_anual_df = pd.read_csv("data/" + station + ".csv", sep=";")

    if preproc:
        anual_df = pre_processing(raw_anual_df)

        anual_df["hora"] = anual_df["data_hora"].dt.hour
    else:
        return raw_anual_df

    return anual_df

def load_and_filter(station, start_hour, end_hour):

    anual_df = load_files(station)

    anual_df = filter_between(anual_df, "hora", start_hour, end_hour)

    return anual_df

def data_to_input_and_output_lstm(data, using_standard_scaler=False):
    # VERIFICAR HORIZONTE
    input_data = []
    output_data = []
    for index in range(0, len(data) - janela_tempo):
        input_sample = data['radiacao'][index:index + janela_tempo]
        output_sample = data['radiacao'][index + janela_tempo]

        input_data.append(input_sample)
        output_data.append(output_sample)

    if using_standard_scaler:
        return teste(np.array(input_data)), teste(np.array(output_data))
    else:
        return np.array(input_data)/1000, np.array(output_data)/1000

def data_to_input_and_output_cnn_lstm(df_inpupt, df_target, janela, horizonte=1):
    # VERIFICAR HORIZONTE
    X_clima, X_rad, y = [], [], []
    for i in range(len(df_inpupt) - janela - horizonte + 1):
        dados_cnn = df_inpupt.iloc[i:i+janela].values
        dados_lstm = df_target.iloc[i:i+janela].values
        target = df_target.iloc[i+janela+horizonte-1]

        X_clima.append(dados_cnn)
        X_rad.append(dados_lstm)
        y.append(target)

    return np.array(X_clima), np.array(X_rad), np.array(y)

def get_files(n=1, lazy=True):
    cwd = os.getcwd()

    base_path = "data/"
    dict_data = {}
    list_files = [x for x in listdir(base_path) if ".csv" in x.lower()]

    if n:
        _range = list_files[0:n]
    else:
        _range = list_files

    for i in _range:
        print("Processando arquivo " + i)
        code = i.split("_")[3]

        cmd = ["head", "-8", cwd+ "/" + base_path + i]
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE)

        result = p.communicate()
        head = result[0].decode('ISO-8859-1')

        if lazy:
            data = base_path + i
        else:
            data = pd.read_csv(base_path+i, sep=";", skiprows=10)

        dict_data[code] = [head, data]

    return dict_data

def getDictToRenameDataFrame(list_columns):
    list_columns_new = []
    for i in list_columns:
        list_columns_new.append(str
              .lower(i)
              .replace(" ", "_")
              . replace("(", "_")
              .replace(")", "")
              .replace("/", "")
              .replace("²", "2")
              .replace("°", "")
              .replace("%", "perc")
              .replace("._", "_")
              .replace(".", "_")
              .replace("__", "_")
              .replace("_-_", "_")
              .replace(",_", "_")
             )

    return dict(zip(list_columns, list_columns_new))

def cols_standardization(df):
    list_columns = df.columns

    list_dict_to_rename = getDictToRenameDataFrame(list_columns)

    return df.rename(columns=list_dict_to_rename).rename(columns={"radiacao_kjm2": "radiacao"})

def hour_transform(n):
    if len(str(n)) == 4:
        return str(n)[0:2] + ":" + str(n)[2:] + ":00"
    elif len(str(n)) == 3:
        return "0" + str(n)[0:1] + ":" + str(n)[1:] + ":00"
    elif n == 0:
        return "00:00:00"

def create_datetime_feature(df):
    timez = timezone("America/Sao_Paulo")

    df["hora_medicao"] = df["hora_utc"].apply(hour_transform)

    df["data_hora_str"] = df["data"] + " " + df["hora_medicao"].astype("str")

    df["data_hora"] = pd.to_datetime(df["data_hora_str"], format="%d/%m/%Y %H:%M:%S")

    df = df.set_index("data_hora")
    df["data_hora"] = df.index.tz_localize(pytz.utc).tz_convert(timez)
    df["data"] = df["data_hora"].dt.date

    df["data_str"] = df["data"].astype("str")
    df["hora"] = df['data_hora'].dt.hour
    return df.drop(["data_hora_str", "hora_utc", "hora_medicao"], axis=1)


def create_split_date_features(df):
    df["dia"] = df["data_hora"].dt.day
    df["mes"] = df["data_hora"].dt.month
    df["ano"] = df["data_hora"].dt.year

    return df

def create_category(column, df):
    labels = ["A", "B", "C", "D"]
    classes = df.describe()[column][3:8].values

    if classes[1] == 0:
        classes[1] = classes[1]+0.1
        print("aqui")
    print(classes)

    return pd.cut(x = df[column],
         bins = classes,
         labels = labels,
         include_lowest = True)

def removeNulls(df, col):
    return df[df[col].notnull()]

def pre_processing(raw_df, is_brt=False):
    df = cols_standardization(raw_df)
    df = create_datetime_feature(df)
    df = create_split_date_features(df)
    df = removeNulls(df, "radiacao")
    df = change_types(df)

    return df

def reduce_df(list_df, key):
    if not key:
        return reduce(lambda x, y: pd.merge(x, y), list_df)

    return reduce(lambda x, y: pd.merge(x, y, on = key), list_df)

def load(path):
    return pd.read_csv(path, sep=";", encoding = "ISO-8859-1", skiprows=8)

def get_perc_nulls(df):
    return (df.isnull().sum()/(len(df)))*100

def get_percentils(df, col):
    for i in range(0, 101):
        value_str = str(i)

        if len(value_str) == 1:
            value_str = "0.0"+value_str

        elif len(value_str) == 2:
            value_str = "0."+value_str
        else:
            value_str = "1.0"

        double_value = float(value_str)
        print(value_str, df[col].quantile(double_value))

def plot_by_col(df, col_grouped, col_target, axs, estacao=None):
    df = df[[col_grouped, col_target]].groupby([col_grouped]).mean().reset_index()

    if axs is None:
      sns.barplot(data=df, x=df[col_grouped], y=df[col_target])

    else:
      axs.set_ylabel(str(estacao))
      sns.barplot(data=df, x=df[col_grouped], y=df[col_target], ax=axs)


def plot_by_range(df, col_x, col_y, dt_start, dt_end):
    df = filter_between(df, col_x, dt_start, dt_end)

    sns.lineplot(data=df, x=df[col_x], y=df[col_y])

def filter_between(df, col, value_1, value_2):
    _filter = (df[col] >= value_1) & (df[col] <= value_2)
    return df[_filter]

def set_plot_size(x, y):
    sns.set_theme(rc={'figure.figsize':(x,y)})

def change_types(df):
    list_columns = df.drop("data_str", axis=1).columns

    for i in list_columns:
        if 'object' in df[i].dtypes.name:
            try:
                df[i] = df[i].str.replace(",", ".").astype('float64')
            except:
                continue

        else:
            continue


    return df

def treat_columns(df):

    columns_to_drop = ['pressão_atmosferica_max_na_hora_ant_aut_mb',
                       'pressão_atmosferica_min_na_hora_ant_aut_mb',
                       'temperatura_do_ponto_de_orvalho_c',
                       'temperatura_máxima_na_hora_ant_aut_c',
                       'temperatura_mínima_na_hora_ant_aut_c',
                       'temperatura_orvalho_max_na_hora_ant_aut_c',
                       'temperatura_orvalho_min_na_hora_ant_aut_c',
                       'umidade_rel_max_na_hora_ant_aut_perc',
                       'umidade_rel_min_na_hora_ant_aut_perc',
                      'vento_direção_horaria_gr__gr',
                      'vento_rajada_maxima_ms']

    columns_to_rename = {'precipitação_total_horário_mm': 'precipitacao',
                        'pressao_atmosferica_ao_nivel_da_estacao_horaria_mb': 'press_atmo',
                        'temperatura_do_ar_bulbo_seco_horaria_c': 'temperatura',
                        'umidade_relativa_do_ar_horaria_perc': 'umidade',
                         'vento_velocidade_horaria_ms':'vento_velocidade_horaria'}

    return df.rename(columns=columns_to_rename).drop(columns_to_drop, axis=1)

def my_autocov(df, col, interval=1):
    serie = np.array(df[col].to_list())

    # serie = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

    xt = serie[:-interval]
    xt1 = serie[interval:]
    n_pairs = len(xt)

    mean_xt = xt.sum(axis=0)/n_pairs
    mean_xt1 = xt1.sum(axis=0)/n_pairs

    dev_xt = xt - mean_xt
    dev_xt1 = xt1 - mean_xt1

    d = {"xt": xt, "xt1": xt1}
    df_new = pd.DataFrame(data=d)

    sns.scatterplot(df_new, x="xt", y="xt1")

    return dev_xt.dot(dev_xt1)/n_pairs

def plot_distrib_horario(df, axs, estacao=None):
    local_anual_df = df[["data_hora", "radiacao", "temp_ins_c"]]
    local_anual_df["hora"] = local_anual_df["data_hora"].dt.hour

    plot_by_col(local_anual_df, "hora", "radiacao", axs, estacao)

def plot_results(df_orig, pred, real):
    set_plot_size(12, 6)

    period = df_orig[0:len(df_orig) - janela_tempo]['data']
    pred = pred.reshape(pred.shape[0])

    df_pred = pd.DataFrame(data=pred, index=period, columns=["predicted"])
    df_real = pd.DataFrame(data=real, index=period, columns=["real"])

    sns.lineplot(df_pred, palette=["red"], ci=None)
    sns.lineplot(df_real, palette=["blue"], ci=None)

def plot_results_2(pred, real, period):
    set_plot_size(12, 6)

    pred = pred.reshape(pred.shape[0])

    df_pred = pd.DataFrame(data=pred, index=period, columns=["predicted"])
    df_real = pd.DataFrame(data=real, index=period, columns=["real"])

    sns.lineplot(df_pred, palette=["red"], ci=None)
    sns.lineplot(df_real, palette=["blue"], ci=None)

def get_metrics(test_pred, y_val):
    mse = mean_squared_error(test_pred.reshape(test_pred.shape[0]), y_val)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(test_pred.reshape(test_pred.shape[0]), y_val)
    r2 = r2_score(test_pred.reshape(test_pred.shape[0]), y_val)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2}

def normalize_leo(X):
    scaler = MinMaxScaler()
    return scaler.fit_transform(X)

def teste(values):
    scaler = StandardScaler()
    if len(values.shape) == 1:
        values = values.reshape(-1, 1)

    scaler = scaler.fit(values)
    # print('Mean: %f, StandardDeviation: %f' % (scaler.mean_, math.sqrt(scaler.var_)))
    # normalize the dataset and print
    standardized = scaler.transform(values)

    return standardized

def integrated_gradients(model,
                          input_tensor,
                          target_class_idx=None,
                          baseline=None,
                          m_steps=100):
    """
    Compute Integrated Gradients for a Keras/TensorFlow model.
    """
    if baseline is None:
        baseline = np.zeros_like(input_tensor).astype(np.float32)

    input_tensor = input_tensor.astype(np.float32)
    baseline = baseline.astype(np.float32)

    interpolated_inputs = [
        baseline + (float(alpha) / m_steps) * (input_tensor - baseline)
        for alpha in range(0, m_steps + 1)
    ]
    interpolated_inputs = tf.convert_to_tensor(np.concatenate(interpolated_inputs, axis=0))

    with tf.GradientTape() as tape:
        tape.watch(interpolated_inputs)
        preds = model(interpolated_inputs)

        if target_class_idx is not None:
            outputs = preds[:, target_class_idx]
        else:
            outputs = tf.reduce_sum(preds, axis=1)  # Regressão

    grads = tape.gradient(outputs, interpolated_inputs)

    avg_grads = tf.reduce_mean(grads[:-1], axis=0)  # Exclui último ponto

    integrated_grads = (input_tensor - baseline) * avg_grads.numpy()

    return integrated_grads.squeeze()

def get_feature_importance(X_train, Y_train, cols, n_features):

    time_steps = 11
    # n_features = 16

    model = Sequential([
        Input(shape=(n_features, 1)),
        Conv1D(32, kernel_size=2, activation='relu'),
        Conv1D(16, kernel_size=2, activation='relu'),
        GlobalAveragePooling1D(),
        Dense(1, activation='linear')  # Para regressão
    ])

    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, Y_train, epochs=10, verbose=0)

    X_sample = X_train[0:1]

    attributions = integrated_gradients(model, X_sample, m_steps=100)


    d = {"feature": cols, "value": attributions}

    local_df = pd.DataFrame(data=d)
    plt.xticks(rotation=90)
    sns.barplot(local_df, x="feature", y="value")

def create_table_results(station, nn_type, results):

    cols = list(results[0]["metrics"].keys())

    list_metrics = np.transpose([[results[x]["metrics"][y] for x in results] for y in cols])

    index_times = list_metrics.shape[0]

    df = pd.DataFrame(
        list_metrics,
        columns=[cols]
    )

    arrays = [
        [station for x in range(index_times)],
        [nn_type for x in range(index_times)],
        [str(x+1) for x in range(index_times)]]

    tuples = list(zip(*arrays))

    index = pd.MultiIndex.from_tuples(tuples, names=["estacao", "tipo_rede", "execucao"])

    return df.set_index(index)

def create_table_results2(station, results):
    cols = list(results.keys())

    list_metrics = np.transpose([[results[y]] for y in cols])
    df = pd.DataFrame(
          list_metrics,
          columns=[cols]
      )

    df["station"] = station

    return df

def adfuller_test(df):
    result=ts.adfuller(df)
    labels = ['Teste estatístico ADF','p-valor','Num Lags','Numero de observações']
    for value, label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("Fortes evidências contra a hipotese nula(Ho), ou seja, pode ser rejeitada. Os dados não possuem uma raíz unitária, portanto tem estacionáriedade.")
    else:
        print("Fracas evidências contra a hipotese nula(Ho), a série temporal possui uma raíz unitária, indicando que é não estacionária.")


def get_seasonal_interval(df, start_hour):
    serie = df["hora"]
    indices = serie[serie == start_hour].index.to_list()

    if len(indices) < 2:
        print("O valor aparece menos de duas vezes.")
        return None

    indices_posicionais = [serie.index.get_loc(i) for i in indices]

    return indices_posicionais[1]

def calculate_acf_pacf(df):
    df_acf_pacf = df[["data_hora", "radiacao"]].set_index("data_hora")

    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = sm.graphics.tsa.plot_acf(df_acf_pacf,lags=40, ax=ax1)
    ax2 = fig.add_subplot(212)
    fig = sm.graphics.tsa.plot_pacf(df_acf_pacf,lags=40, ax=ax2)

def filter_seasons(df, date_col):
    outono = ((df[date_col] >= "2023-03-20") & (df[date_col] < "2023-06-20")) | ((df[date_col] >= "2024-03-20") & (df[date_col] < "2024-06-20"))
    inverno = ((df[date_col] >= "2023-06-20") & (df[date_col] < "2023-09-22")) | ((df[date_col] >= "2024-06-20") & (df[date_col] < "2024-09-22"))
    primavera = ((df[date_col] >= "2023-09-22") & (df[date_col] < "2023-12-21")) | ((df[date_col] >= "2024-09-22") & (df[date_col] < "2024-12-21"))
    verao = ((df[date_col] >= "2023-12-21") & (df[date_col] < "2024-03-20")) | ((df[date_col] >= "2024-12-21") & (df[date_col] < "2025-03-20"))

    return df[outono], df[inverno], df[primavera], df[verao]
